# K-최근접이웃 (KNN)
| 유사한 특성을 가진 데이터는 유사한 범주에 속하는 경향이 있다는 가정으로 데이터를 분류하는 알고리즘(지도학습!)
1. KNN Classification
* 학습데이터를 그대로 저장한다
* 새로운 데이터 포인트에 대해 가장 가까운 k개의 데이터 포인트를 찾는다 (주로 유클리디안 거리 사용)
    * k값에 따라 새로운 데이터의 라벨링이 달라질 수 있다
* 그것들로 부터 새로운 데이터 포인트의 범주를 라벨링한다

**손글씨, 위성 이미지 분석 등에서 활용**

**비모수적 방식**을 사용하여 결정경계가 매우 불규칙한 분류 상황에서 종종 높은 예측 성능을 보임
2. KNN Regression
* 가까운 이웃 데이터를 이용하여 회귀선을 도출한다
* 하나의 회귀식을 도출하는 것이 아니라는 점에서 선형 회귀와 차이를 가진다. 
    * 때문에 종속변수에 대한 독립변수의 영향력을 의미하는 회귀계수는 KNN에서 확인되지 않는다.

## KNN 분류
```sklearn.neighbors.KNeighborsClassifier(n_neighbors, weights, algorithm, leaf_size, p, metric, metric_params, n_jobs)```
* n_neighbors: 쿼리에 사용한 이웃 k의 수
* weights: 예측에 사용되는 가중치 함수
    - uniform: 동일한 가중치 부여
    - distance: 거리의 역수로 가중치 부여
* algorithm: 이웃 데이터를 계산하는 데 사용되는 알고리즘 'auto'로 설정하면 fit()에 전달된 값을 기반으로 자동 설정
* leaf_size: Tree에서 몇 대 몇으로 뻗어나갈지 나타내는 값. 작을수록 noise가 생기고 클수록 성능이 떨어짐
* p: minkowski 메트릭에 대한 검정력 매개변수, 2일때 유클리디안 사용하는 것
* metric: 거리 메트릭 기본은 민코우스키
* metric_params: 메트릭 함수에 대한 추가 키워드 인수
* n_jobs: 근접 데이터 탐색을 위한 병령 작업의 수

### KNN_분류 예제

In [2]:
# 간 질환자 데이터 예제
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import *

In [3]:
# 데이터 로드
liver = pd.read_csv('https://raw.githubusercontent.com/ADPclass/ADP_book_ver01/main/data/indian_liver_patient.csv')
liver.head()

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
0,65,Female,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,Male,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,Male,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,Male,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,Male,3.9,2.0,195,27,59,7.3,2.4,0.40,1


In [4]:
# 성별 변수 인코딩 (OneHot)
liver.Gender = np.where(liver.Gender=='Female', 0, 1) #T이면 0, F면 1

In [5]:
liver.isna().sum()

Age                           0
Gender                        0
Total_Bilirubin               0
Direct_Bilirubin              0
Alkaline_Phosphotase          0
Alamine_Aminotransferase      0
Aspartate_Aminotransferase    0
Total_Protiens                0
Albumin                       0
Albumin_and_Globulin_Ratio    4
Dataset                       0
dtype: int64

In [6]:
# 타겟 비율 확인
liver.Dataset.value_counts(normalize=True)

Dataset
1    0.713551
2    0.286449
Name: proportion, dtype: float64

In [7]:
# 결측치 제거
liver.dropna(axis=0, inplace=True)
# 데이터 분할
X = liver.drop('Dataset', axis=1)
y = liver['Dataset']
x_train, x_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=.3, random_state=1)
# 학습
clf = KNeighborsClassifier(n_neighbors=15, weights='uniform')
clf.fit(x_train, y_train)
# 성능
pred_tt = clf.predict(x_test)

# 혼동행렬
cm = confusion_matrix(y_test, pred_tt)
acc = accuracy_score(y_test, pred_tt)
rcll = recall_score(y_test, pred_tt)
prc = precision_score(y_test, pred_tt)
f1 = f1_score(y_test, pred_tt)

print(cm)
print(f'''
      acc: {acc}%
      recall: {rcll}
      precision: {prc}
      f1_score: {f1}''')

[[106  18]
 [ 39  11]]

      acc: 0.6724137931034483%
      recall: 0.8548387096774194
      precision: 0.7310344827586207
      f1_score: 0.788104089219331


## KNN 회귀
```sklearn.neighbor.KNeighborRegressor(분류기와 파라미터 동일)```

### KNN_회귀 예제

In [8]:
# 보험료 데이터
data = pd.read_csv('https://raw.githubusercontent.com/ADPclass/ADP_book_ver01/main/data/insurance.csv')
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [9]:
# 원핫인코딩
data.sex = np.where(data.sex=='female',0,1)
data.smoker = np.where(data.smoker=='yes',0,1)
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,0,southwest,16884.92400
1,18,1,33.770,1,1,southeast,1725.55230
2,28,1,33.000,3,1,southeast,4449.46200
3,33,1,22.705,0,1,northwest,21984.47061
4,32,1,28.880,0,1,northwest,3866.85520


In [22]:
corr = data.drop(['region', 'sex', 'smoker'], axis=1)
corr.corr()

,age,bmi,children,charges
age,1.000000,0.109272,0.042469,0.299008
bmi,0.109272,1.000000,0.012759,0.198341
children,0.042469,0.012759,1.000000,0.067998
charges,0.299008,0.198341,0.067998,1.000000


In [11]:
from sklearn.preprocessing import StandardScaler

In [13]:
X.head()

,age,sex,bmi,children,smoker
0,19,0,27.900,0,0
1,18,1,33.770,1,1
2,28,1,33.000,3,1
3,33,1,22.705,0,1
4,32,1,28.880,0,1


In [24]:
# 데이터 분할
X = data.drop(['charges', 'region'], axis=1)
y = data['charges']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=1)

# 연속형 변수 스케일링
sc = StandardScaler()
numeric = ['age', 'bmi', 'children']
x_train[numeric] = sc.fit_transform(x_train[numeric])
x_test[numeric] = sc.transform(x_test[numeric])

# 학습 (가중치 옵션 비교)
clr_uni = KNeighborsRegressor(n_neighbors=20, weights='uniform')
clr_dis = KNeighborsRegressor(n_neighbors=20, weights='distance')

clr_uni.fit(x_train, y_train)
clr_dis.fit(x_train, y_train)
# 예측
pred_uni = clr_uni.predict(x_test)
pred_dis = clr_dis.predict(x_test)
# 비교
def compare(preds):  
    result = []
    for pred in preds:
        mae = mean_absolute_error(y_test, pred).round(3)
        mse = mean_squared_error(y_test, pred).round(3)
        rmse = np.sqrt(mse).round(3)
        result.append([mae, mse, rmse])
    return pd.DataFrame(result, index=['uniform', 'distance'], columns=['mae', 'mse', 'rmse'])

compare_df = compare([pred_uni, pred_dis])
compare_df


,mae,mse,rmse
uniform,3885.032,4.196391e+07,6477.956
distance,3621.707,3.765837e+07,6136.642
